## Gemini Pro Function Call

Feedback : shins777@gmail.com. 

* 이 Colab 은 Google Gemini Pro 사용시 Function call 기능을 구현한 예제입니다.
* 코드는 Gemini Pro API 기반으로 처리하며 그에 따른 API는 아래 링크 참고하세요.  
    * 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [3]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
# from google.colab import auth
# auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [9]:
model_name="gemini-pro"
project="ai-hangsik"
location="asia-northeast3"

#### Google Generative AI를 사용하기 위한 aiplatform initialize.

In [10]:
from google.cloud import aiplatform
aiplatform.init(project=project, location = location)

### Function Call

* LLM 처리중에 외부 함수 실행 할때 필요한 기능으로 Function call을 활용할 수 있습니다.
* Function Call API : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling?hl=ko

#### LLM 처리중에 호출 되는 함수 정의
* 아래 함수는 LLM 처리 과정에서 함수가 호출되면서 처리되는 예제이며, 이 해당 함수의 로직내에서 특정 함수를 호출하거나 또는 외부 REST API를 호출해서 로직을 구성할 수 있습니다.

In [15]:
def get_stock_price(ticker:str) ->str:
  """
  Arg 로 넘어오는 ticker는 LLM에 의해서 찾은 값임.
     - 만일, "Google 주식값을 알려줘"라고 했다면 여기 arg 로 넘어오는 인자는 GOOG 또는 GOOGL이 될수 있음.
  
  이곳에서 주식정보를 호출하기위한 로직을 구현. 아래는 단순하게 constant 값을 리턴.
  """
  print(ticker)
  
  return "Stock price is 445.55"

#### Gemini Function call 함수
* Tool을 사용하여 LLM이 참고하여 사용할 수 있도록 Function을 정의해야 합니다.
* 해당 Tool은 모델이 인식할 수 있도록 호출을 통해서 처리합니다.
* Function handler는 여러개의 function이 있을 때 선택적으로 처리하도록 구성합니다. 

In [16]:
from vertexai.preview.generative_models import FunctionDeclaration
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.generative_models import Part
from vertexai.preview.generative_models import Tool

def function_call(prompt:str)->str:
  
  # Tools
  tools = Tool(function_declarations=[
      FunctionDeclaration(
          name="get_stock_price",
          description="Get the current stock price of a given company",
          parameters={
              "type": "object",
              "properties": {
                  "ticker": {
                      "type": "string",
                      "description": "Stock ticker symbol"
                  }
              }
          },
      )
  ])

  # Model Initialization
  model = GenerativeModel("gemini-pro",
                          generation_config={"temperature": 0},
                          tools=[tools])

  chat = model.start_chat()

  # Send a prompt to the chat
  response = chat.send_message(prompt)
  function_call = response.candidates[0].content.parts[0].function_call
  
  function_handlers = {
      "get_stock_price": get_stock_price,
  }

  if function_call.name in function_handlers:
      function_name = function_call.name
      chat_response = ""

      args = {key: value for key, value in function_call.args.items()}

      if args:
          function_response = function_handlers[function_name](args)

          part_data = Part.from_function_response(
                  name=function_name,
                  response={
                      "content": function_response,
                  }
          )

          cal_eq = part_data.to_dict()['function_response']['response']['content']

          response = chat.send_message(part_data,)
          chat_response = response.candidates[0].content.parts[0].text

          print(f"Function call 이후 응답: {chat_response}")
          return chat_response
      else:
          print("No arguments found for the function.")
          return "주식정보를 제공해야 할 회사 이름을 알려주세요."
  else:
      print("Function call 이 되지 않은 응답")
      return response.text


#### Function call 이 되는 prompt로 호출
* 이 경우는 Prompt 내용을 기반으로 LLM이 판단하여 현재 등록된 함수를 호출해야 할 경우, 해당 함수를 호출해서 결과를 LLM이 받아서 해당 값을 생성해서 리턴해줍니다.

In [17]:
prompt = "구글의 주식 가격이 현재 얼마인가요 ?"

function_call(prompt)

{'ticker': 'GOOG'}
Function call 이후 응답: 현재 구글의 주식 가격은 445.55입니다.


'현재 구글의 주식 가격은 445.55입니다.'

#### Function call 이 호출되지 않는 Prompt로 호출
* 이 경우는 Prompt 내용에 대해서 LLM이 판단하여 호출될 함수가 없을때의 경우입니다.

In [18]:
prompt = "오늘의 날씨는 어떤가요 ?"

function_call(prompt)

Function call이 되지 않은 응답


'죄송합니다. 해당 요청을 충족할 수 없습니다. 날씨 정보를 제공하는 도구가 없습니다.'